# Titanic



**Proposta:** Construir um modelo de previsão que responda a pergunta: "Quais tipos de pessoas tinham maior probabilidade de sobreviver?" usando os dados dos passageiros (como nome, idade, gênero, classe socio-econômica, etc...).

**Link do Kaggle:** https://www.kaggle.com/competitions/titanic


## Análise, tratamento e pré-processamento dos dados de treino


Link do Kaggle: https://www.kaggle.com/competitions/titanic

In [ ]:
import numpy as np
import pandas as pd
dados_teste = pd.read_csv('/content/drive/MyDrive/CursoML/Titanic/test.csv',sep=',', encoding='iso-8859-1')
dados_treino = pd.read_csv('/content/drive/MyDrive/CursoML/Titanic/train.csv',sep=',', encoding='iso-8859-1')

In [ ]:
dados_treino

In [ ]:
dados_treino.info()

In [ ]:
dados_treino.isnull().sum()

In [ ]:
pd.pivot_table(dados_treino, index='Survived', values = ['Age','SibSp','Parch','Fare'])

In [ ]:
dados_treino2 = dados_treino.dropna()

In [ ]:
dados_treino['Age'].fillna(dados_treino2['Age'].mean(), inplace=True)

Ao invés de dropar a categoria Cabin, buscar padrões como a compra de tickets de várias cabines e das letras das cabines

In [ ]:
dados_treino.Cabin

In [ ]:
dados_treino['cabin_quantidade'] = dados_treino.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
dados_treino['cabin_quantidade'].value_counts()

In [ ]:
dados_treino['cabin_letra'] = dados_treino.Cabin.apply(lambda x: str(x)[0])
dados_treino['cabin_letra'].value_counts()

Ticket: Separar por número de letras

In [ ]:
dados_treino['ticket_numerico'] = dados_treino.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
dados_treino['ticket_letras'] = dados_treino.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

In [ ]:
dados_treino['ticket_numerico'].value_counts()

In [ ]:
dados_treino['ticket_letras'].value_counts()

In [ ]:
pd.pivot_table(dados_treino,index='Survived',columns='ticket_numerico', values = 'Ticket', aggfunc='count')

In [ ]:
pd.pivot_table(dados_treino,index='Survived',columns='ticket_letras', values = 'Ticket', aggfunc='count')


Nome: Manter apenas os títulos

In [ ]:
dados_treino.Name.head(50)
dados_treino['name_titulo'] = dados_treino.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
dados_treino['name_titulo'].value_counts()

In [ ]:
dados_treino.drop(['PassengerId'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
dados_treino['Fare'] = StandardScaler().fit_transform(dados_treino['Fare'].values.reshape(-1, 1))

In [ ]:
dados_treino

In [ ]:
#conta quantas cabines cada passageiro tem
dados_treino['cabin_quantidade'] = dados_treino.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

In [ ]:
#guarda apenas a primeira letra das cabines
dados_treino['cabin_letra'] = dados_treino.Cabin.apply(lambda x: str(x)[0])

In [ ]:
#classifica se é ou não numérico
dados_treino['ticket_numerico'] = dados_treino.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)

In [ ]:
#junta as letras do ticket
dados_treino['ticket_letras'] = dados_treino.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

In [ ]:
#guarda apenas os títulos
dados_treino['name_titulo'] = dados_treino.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
dados_treino['Embarked'].value_counts()

In [ ]:
dados_treino['Sex'].replace({'male':0, 'female': 1}, inplace=True)

In [ ]:
dados_treino['Embarked'].fillna(dados_treino['Embarked'].mode()[0], inplace=True)

In [ ]:
dados_treino['Embarked'].replace({'S':0, 'C': 1, 'Q':2}, inplace=True)

In [ ]:
dados_treino = dados_treino.drop(['Name'], axis=1)

In [ ]:
dados_treino = dados_treino.drop(['Ticket'], axis=1)

In [ ]:
dados_treino = dados_treino.drop(['Cabin'], axis=1)

In [ ]:
# como as letras não têm tantas similaridades, não vale a pena manter essa coluna
dados_treino.drop(['ticket_letras'], axis=1, inplace=True)

In [ ]:
previsores_treino = dados_treino.iloc[:, 1:12].values

In [ ]:
previsores_treino

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

colunas_categoricas = [8, 10]

col_transformer = ColumnTransformer(
    transformers=[('OneHot', OneHotEncoder(handle_unknown='ignore'), colunas_categoricas)],
    remainder='passthrough'
)

previsores_treino2 = col_transformer.fit_transform(previsores_treino)

scaler = StandardScaler()

previsores_treino3 = scaler.fit_transform(previsores_treino2)

In [ ]:
previsores_treino2

In [ ]:
previsores_treino3 = StandardScaler().fit_transform(previsores_treino2)

In [ ]:
previsores_treino3

In [ ]:
alvo = dados_treino.iloc[:, 0].values

## Análise, tratamento e pré-processamento dos dados de teste


In [ ]:
dados_teste

In [ ]:
dados_teste.isnull().sum()

In [ ]:
dados_teste['cabin_quantidade'] = dados_teste.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

In [ ]:
dados_teste['cabin_letra'] = dados_teste.Cabin.apply(lambda x: str(x)[0])

In [ ]:
dados_teste['ticket_numerico'] = dados_teste.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)

In [ ]:
dados_teste['ticket_letras'] = dados_teste.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1]) >0 else 0)

In [ ]:
dados_teste['name_titulo'] = dados_teste.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
dados_teste['Age'].fillna(dados_teste['Age'].mean(), inplace=True)

In [ ]:
dados_teste['Fare'].fillna(dados_teste['Fare'].mean(), inplace=True)

In [ ]:
dados_teste['Fare'] = StandardScaler().fit_transform(dados_teste['Fare'].values.reshape(-1, 1))


In [ ]:
dados_teste = dados_teste.drop(['Name'], axis=1)

In [ ]:
dados_teste = dados_teste.drop(['Cabin'], axis=1)

In [ ]:
dados_teste = dados_teste.drop(['Ticket'], axis=1)

In [ ]:
dados_teste = dados_teste.drop(['PassengerId'], axis=1)

In [ ]:
dados_teste['Sex'].replace({'male':0, 'female': 1}, inplace=True)

In [ ]:
dados_teste['Embarked'].replace({'S':0, 'C': 1, 'Q':2}, inplace=True)

In [ ]:
dados_teste.dtypes

In [ ]:
dados_teste.drop(['ticket_letras'], axis=1, inplace=True)

In [ ]:
dados_teste2 = col_transformer.transform(dados_teste)

In [ ]:
dados_teste2

In [ ]:
dados_teste3 = scaler.transform(dados_teste2)

In [ ]:
dados_teste3

## Aplicação dos modelos


SVM:

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf', random_state=1, C = 2)
svm.fit(previsores_treino3, alvo)

previsoes_svm = svm.predict(dados_teste3)

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits = 30, shuffle=True, random_state = 5)
modelo = SVC(kernel='rbf', random_state=1, C = 2)
resultado = cross_val_score(modelo, previsores_treino3, alvo, cv = kfold)

print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Naive Bayes:

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
resultado = cross_val_score(gnb,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Regressão Logística:

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 2000)
resultado = cross_val_score(lr,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

KNN:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=1)
resultado = cross_val_score(knn,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Árvore de decisão:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
arvore = DecisionTreeClassifier(criterion='entropy', random_state = 0, max_depth=3)
resultado = cross_val_score(arvore,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

Random Forest:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random = RandomForestClassifier(n_estimators=150, criterion='entropy', random_state = 0, max_depth=4)
resultado = cross_val_score(random,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

XGBoost:

In [ ]:
from xgboost import XGBClassifier
xg = XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=250, objective='binary:logistic', random_state=3)
resultado = cross_val_score(xg,previsores_treino3,alvo,cv=5)
print("Acurácia Média: %.2f%%" % (resultado.mean() * 100.0))

#### Entre os algoritmos testados, o **XGBoost** apresentou a maior acurácia.